In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("s076923/pytorch-transformer")

print("Path to dataset files:", path)

Mounting files to /kaggle/input/pytorch-transformer...
Path to dataset files: /kaggle/input/pytorch-transformer


In [5]:
import os
import torch
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset

In [ ]:
/kaggle/input/pytorch-transformer/datasets/coco/train
/kaggle/input/pytorch-transformer/datasets/coco/val
/kaggle/input/pytorch-transformer/datasets/coco/annotations/train_annotations.json
/kaggle/input/pytorch-transformer/datasets/coco/annotations/val_annotations.json

In [8]:
root = "/kaggle/input/pytorch-transformer/datasets/coco/"
os.path.join(root, 'anno', 'test')

'/kaggle/input/pytorch-transformer/datasets/coco/anno/test'

In [9]:
train = True
directory = "train" if train else "val"
annotations = os.path.join(root, "annotations", f"{directory}_annotations.json")

In [12]:
coco = COCO(annotations)

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [22]:
coco.loadAnns(coco.getAnnIds(495357))

[{'segmentation': [[374.46,
    310.42,
    386.68,
    310.17,
    394.32,
    308.64,
    398.4,
    303.54,
    397.38,
    301.0,
    401.71,
    301.51,
    403.49,
    296.16,
    402.98,
    291.32,
    402.22,
    286.48,
    400.95,
    283.42,
    394.32,
    281.13,
    391.78,
    277.06,
    388.98,
    272.47,
    387.7,
    270.94,
    382.1,
    270.18,
    378.53,
    267.89,
    374.46,
    263.81,
    368.35,
    260.76,
    366.05,
    256.94,
    363.25,
    253.88,
    358.67,
    251.08,
    357.14,
    247.51,
    353.83,
    245.22,
    351.03,
    244.46,
    345.93,
    244.97,
    342.88,
    248.02,
    341.35,
    251.08,
    339.57,
    253.63,
    337.02,
    255.15,
    338.8,
    259.23,
    339.41,
    266.1,
    346.51,
    265.74,
    349.35,
    269.3,
    342.25,
    272.49,
    339.41,
    275.69,
    343.67,
    277.11,
    348.29,
    274.27,
    353.61,
    276.4,
    357.17,
    283.86,
    357.88,
    292.38,
    355.75,
    301.62,
    352.

In [23]:
class COCODataset(Dataset):
    def __init__(self, root, train, transform=None):
        super(self, COCODataset).__init__()
        directory = "train" if train else "val"
        annotations = os.path.join(root, "annotations", f"{directory}_annotations.json")
        self.coco = COCO(annotations)
        self.image_path = os.path.join(root, directory)
        self.transform = transform 
        self.categories = self.__get_categories()
        self.data = self.__load_data()
           
    def __get_categories(self):
        categories = {0 : 'background'}
        for category in self.coco.cats.values():
            categories[category['id']] = category['name']
        return categories
    
    def __load_data(self):
        data = []
        for _id in self.coco.imgs:
            file_name = self.coco.loadImgs(_id)[0]['file_name']
            image_path = os.path.join(self.image_path, file_name)
            image = Image.open(image_path).convert("RGB")

            boxes = []
            labels = []
            anns = self.coco.loadAnns(self.coco.getAnnIds(_id))

            for ann in anns:
                x, y, w, h = ann['bbox']
                boxes.append([x, y, x+w, y+h])
                labels.append(ann['category_id'])
            target = {
                'image_id' : torch.LongTensor([_id]),
                'boxes' : torch.FloatTensor(boxes),
                'labels' : torch.LongTensor(labels)
            }
            data.append([image, target])
        return data
       
    
    def __getitem__(self, index):
        image, target = self.data[index]
        if self.tranform:
            image =self.transform(image)
        return image, target
        
    def __len__(self):
        return len(self.data)

In [ ]:
from torchvision